<h2>Alhambra as validation</h2>

-- Check Alhambra photo-z's meet WL requirement

-- Weight color/may of spec-z to alhambra 

-- train machine on spec-z

-- cull alhambra space until predictions agree.


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
import numpy as np
from weighted_kde import gaussian_kde as gss_kde
import sys 
import time
import cPickle as pickle
import sml_mla as ml

import ml_metrics as ml_m
sys.path.append('/Users/hoyleb/Documents/python/modules/photoz-wg/validation/')
import bh_photo_z_validation as pval
import ml_params as mlp
from astropy.table import Table, vstack

from weighted_kde import gaussian_kde as gss_kde
from scipy.stats import entropy
from sklearn.metrics import r2_score

import math
from scipy.stats import norm
from scipy.special import erf
from sklearn import tree
import copy
from scipy import optimize
from sklearn.neighbors import KNeighborsRegressor, RadiusNeighborsRegressor, NearestNeighbors
import matplotlib.mlab as mlab
from sklearn.neighbors import KernelDensity

import calculate_weights as cw
import kmeansClusters as km
import anomaly_detection as ad
import augment_data as augd
from scipy.stats import entropy

def normalise(arr):
    arr = np.array(arr, dtype=float)
    arr = arr / np.sum(arr)
    return arr



In [ ]:
#how many machines to train
Nmachines = 50

#some functions to generate a machine and to test performance
def rand_machine():
    clf = AdaBoostRegressor(base_estimator=DecisionTreeRegressor())
    if np.random.uniform() < 0.5:
        clf = RandomForestRegressor()
    # get parameters from this MLA
    curr_params = clf.get_params()
    params = mlp.getRandomParams()
    params['n_estimators'] = np.random.randint(10, 100)
    curr_params = ml.combineParameters(curr_params, params)
    curr_params = ml.tree_loss(curr_params, clf)
    clf.set_params(**curr_params)
    return clf

def get_score(truth_, pred_, sample_weight=None):
    return  {'r2' : r2_score(truth_, pred_, sample_weight=sample_weight),
     'other': ml_m.getStats((truth_ - pred_) / (1 + truth_), sample_weight=sample_weight)
     }

#loop over all trees to get a redshift dist
def allPredictions(cl, inp):
    pred = cl.predict(inp)

    pdf = np.zeros((len(pred), len(cl.estimators_)))
    if hasattr(cl, 'estimator_weights_'):
        weights = cl.estimator_weights_ / np.sum(cl.estimator_weights_)
        weightsInt = np.array(len(weights) * 3 / np.sum(weights) * weights, dtype=int)
        wpdf = np.zeros((len(pred), np.sum(weightsInt)))
        del pdf
    sm = 0
    for i, DT in enumerate(cl.estimators_):
        if isinstance(DT, (list, tuple, np.ndarray)):
            DT = DT[0]
        DTz = DT.predict(inp)

        if hasattr(cl, 'estimator_weights_'):
            #make a weighted pdf
            for j in range(weightsInt[i]):
                wpdf[:, sm] = DTz
                sm += 1
        else:
            pdf[:, i] = DTz

    if hasattr(cl, 'estimator_weights_'):
        rind = np.random.choice(range(sm), size=len(pred), replace=True)
        return np.std(wpdf[:, 0:sm], axis=1), np.array([wpdf[i,rind[i]] for i in range(len(rind))])
    
    rind = np.random.choice(range(len(pdf[0])), size=len(pdf), replace=True)
    return np.std(pdf, axis=1), np.array([pdf[i,rind[i]] for i in range(len(rind))])


# parralelisation function
# beware some global variables like score_metric is in here
def xvFitPredict(_clf, inputs, outputs, train_weights_arr, _indho):

    # train on non held out
    _intr = np.setdiff1d(np.arange(len(inputs)), _indho, assume_unique=True)

    inp_ = copy.deepcopy(inputs[_intr])
    outp_ = copy.deepcopy(outputs[_intr])

    #if we have training weights, then bootstrap resample from weights

    wt_ = train_weights_arr[_intr]
    wt_ = wt_ / np.sum(wt_)

    _clf.fit(inp_, outp_, wt_)

    dictXv = {}
    # calculate stats on held out fraction
    dictXv['predict'] = _clf.predict(inputs[_indho])

    dictXv['sigma'] = allPredictions(_clf, inputs[_indho])

    # calcaulate score
    dictXv['scoreXv'] = get_score(outputs[_indho], dictXv['predict'], 
                                  sample_weight=train_weights_arr[_indho])
    dictXv['outputs'] = outputs[_indho]
    dictXv['index'] = _indho
    return dictXv


import augment_data as ad

#kde_augment or kde_knn_augment

def k_fold_val(inputs, outputs, train_weights, Nfolds, Niterations):
      
    indHo = np.array_split(np.arange(len(inputs)), Nfolds)
    results = {}
    score_best = 100
    for i in range(Niterations):
        
        clf = rand_machine()
        resXv = {}
        for j in range(len(indHo)):
            resXv[j] = xvFitPredict(clf, inputs, outputs, train_weights, indHo[j])
            
        scoreXv = np.zeros(Nfolds).astype(float)
        predict = np.zeros(len(inputs), dtype=float)
        sigma = np.zeros(len(inputs), dtype=float)
        for k in resXv:
            scoreXv[k] = resXv[k]['scoreXv']['other']['sigma_68']
            predict[indHo[k]] = resXv[k]['predict']
            sigma[indHo[k]] = resXv[k]['sigma']
            
        score = np.mean(scoreXv)
        print score, np.std(scoreXv)
        
        if score < score_best:
            score_best = score
            #results['xval'] = resXv
            clf.fit(inputs, outputs, train_weights)
            results['clf'] = clf
            results['predict'] = predict
            results['sigma'] = sigma
            results['z_mc'] = predict + np.random.normal(size=len(predict)) * sigma
            
    return results, score_best
            

In [ ]:
almost_black = '#262626'
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams.update({'font.size': 32, 
                     'axes.linewidth': 5,
                    'text.color': almost_black,
                    'xtick.major.size': 4,
                    'ytick.major.size': 4,
                    'legend.fancybox': True,
                    'figure.dpi': 300,
                    'legend.fontsize': 14,
                    'legend.framealpha': 0.8,
                    'legend.shadow': True,
                    'xtick.labelsize': 32,
                     'markersize': 10,
                     'elinewidth':2,
                    'ytick.labelsize': 32})

In [ ]:
imag = np.linspace(17,24,15)
imag

<h3>Does alahmbra photo-z matched to spec-z meet WL requirements?</h3>

In [ ]:
d = Table.read('/Users/hoyleb/DATA/DES/PHOTOZ/Y1Spec_Matched_Alhambra.fits')
#print d.keys()
PRIMUS = np.array([i.replace(' ','')== 'PRIMUS' for i in d['SOURCE']])

i = np.array(d['FLAG'] != 3) * np.array(PRIMUS == False) * (d['MAG_AUTO_I']<24)
d = d[i]
plt.plot(d['zb_1'], d['SPEC_Z'], 'o')
plt.xlabel('Alhambra q-z')
plt.ylabel('spec-z')
deltaz = ml_m.DeltaZ(np.array(d['SPEC_Z']), np.array(d['zb_1']))
print 'stats on (spec-z - alh-z)/(1 + spec-z)'
print ml_m.getStats(deltaz)
print 'for {:} objects'.format(len(deltaz))

print "np.amin(d['MAG_AUTO_I']), np.amax(d['MAG_AUTO_I'])"
print np.amin(d['MAG_AUTO_I']), np.amax(d['MAG_AUTO_I'])

num = []
f = plt.figure()
bns = np.linspace(-1,1, 100)
bn = (bns[1:] + bns[:-1]) / 2.0
i_err = {'cosmos': {}, 'alhambra':{},'i_mag':imag, 'cosmos_num': {},'alhambra_num':{} }


for i in range(len(imag)-1):
    ind = np.array(d['MAG_AUTO_I'] <= imag[i + 1]) * np.array(d['MAG_AUTO_I'] > imag[i])
    num.append(np.sum(ind))
    if np.sum(ind) > 1:
        deltaz = ml_m.DeltaZ(d['SPEC_Z'][ind], d['zb_1'][ind])
        h = np.histogram(deltaz, bins=bns)[0]
        sts = ml_m.getStats(deltaz)
        plt.plot(bn, h, label='{:}'.format(sts['sigma_68']))
        i_err['alhambra']['{:}'.format(imag[i])] = deltaz
        i_err['alhambra_num']['{:}'.format(imag[i])] = np.sum(ind)
        print imag[i + 1]
        print sts
        print np.sum(ind)

<h3>Quantify the redshift error from using photo-z not spec-z as true z</h3>

In [ ]:
#cosmos data
d = Table.read('/Users/hoyleb/DATA/DES/PHOTOZ/Y1Spec_Matched_COSMOS.fits')
PRIMUS = np.array([i.replace(' ','')== 'PRIMUS' for i in d['SOURCE']])
i = np.array(d['FLAG'] != 3) * np.array(PRIMUS == False)
d = d[i]

plt.plot(d['ZMINCHI2'], d['SPEC_Z'], 'o')
plt.xlabel('Cosmos p-z')
plt.ylabel('spec-z')
deltaz = ml_m.DeltaZ(d['SPEC_Z'], d['ZMINCHI2'])
print 'stats on (spec-z - cos-z)/(1 + spec-z)'
print ml_m.getStats(deltaz)
print 'for {:} objects'.format(len(deltaz))

num = []
f = plt.figure()
for i in range(len(imag)-1):
    ind = np.array(d['MAG_AUTO_I_2'] <= imag[i + 1]) * np.array(d['MAG_AUTO_I_2'] > imag[i])
    num.append(np.sum(ind))
    if np.sum(ind) > 1:
        deltaz = ml_m.DeltaZ(np.array(d['SPEC_Z'][ind]), np.array(d['ZMINCHI2'][ind]))
        h = np.histogram(deltaz, bins=bns)[0]
        sts = ml_m.getStats(deltaz)
        plt.plot(bn, h, label='{:}'.format(sts['sigma_68']))
        i_err['cosmos']['{:}'.format(imag[i])] = deltaz
        i_err['cosmos_num']['{:}'.format(imag[i])] = np.sum(ind)
plt.legend()
plt.yscale('log')
f = plt.figure()
plt.plot((imag[1:]+imag[:-1])/2.0, num )


In [ ]:
pickle.dump(i_err, open('/Users/hoyleb/DATA/DES/PHOTOZ/CosmosAlhambra_phot_spec_z_rescaling.p', 'w'))

<h3>Generating photo-z like data with the correct distributions</h3>

In [ ]:
d = d[d['MAG_AUTO_I_2']<24]
ind1 = np.random.choice(np.arange(len(d)), size=1e5, replace=True)


z = np.array(d['SPEC_Z'][ind1]) # or the redshift column
mag_auto_i = np.array(d['MAG_AUTO_I_2'][ind1]) # the magnitude column

#note predictions are only suitable for 16<MAG_AUTO_I<24
z_alh = np.zeros_like(z)
print np.amin(mag_auto_i), np.amax(mag_auto_i)
for i in np.arange(len(i_err['i_mag'])-1):
    ind = (mag_auto_i > i_err['i_mag'][i]) * (mag_auto_i <= i_err['i_mag'][i+1])
    if np.sum(ind) > 0:
        z_alh[ind] = z[ind] +  np.random.choice(i_err['cosmos']['{:}'.format(i_err['i_mag'][i])], size=np.sum(ind), replace=True)

        
plt.plot(z_alh, z, 'o')
plt.xlabel('phot-z')
plt.ylabel('spec-z')

deltaz = ml_m.DeltaZ(z, z_alh)
print 'stats on (spec-z - cos-z)/(1 + spec-z)'
print ml_m.getStats(deltaz)
print " "
print ml_m.getStats(ml_m.DeltaZ(np.array(d['SPEC_Z'][ind1]), np.array(d['ZMINCHI2'])[ind1]))

print " "
print "{'errsig68': 0.0002357407488243309, 'outlierFrac': 3.280067283431455, 'median': 0.00047995645364236903, 'sigma_68': 0.011495836644654334, 'sigma_95': 0.079045752990336296, 'sigma': 0.082332335184199301, 'mean': 0.00061993605824146067}"

In [ ]:
print np.amin(d['MAG_AUTO_I_2']), np.amax(d['MAG_AUTO_I_2'])

In [ ]:
i=6
_ = plt.hist(np.random.choice(i_err['alhambra']['{:}'.format(i_err['i_mag'][i])], size=5000, replace=True), bins=50)

<h3>Compare alahmbra photo-z and cosmos photo-z</h3>

In [ ]:
#compare cosmos and alhambra photo-z's
dac = Table.read('/Users/hoyleb/DATA/DES/PHOTOZ/ALHAMBRA_Matched_COSMOS_1as.fits')
print dac.keys()
#PRIMUS = np.array([i.replace(' ','')== 'PRIMUS' for i in d1['SOURCE']])
i = np.array(dac['MAG_AUTO_I'] < 23.5 ) * np.array(dac['MAG_AUTO_I'] > 17 ) * np.array(dac['MAGERR_AUTO_I'] <0.4 )
#i = np.ones(len(dac), dtype=bool)
plt.plot(dac['ZMINCHI2'][i], dac['zb_1'][i], 'o')
plt.xlabel('Cosmos p-z')
plt.ylabel('ALhambra-z')
deltaz = ml_m.DeltaZ(dac['zb_1'][i], dac['ZMINCHI2'][i])
print 'stats on (alh-z - cos-z)/(1 + spec-z)'
print ml_m.getStats(deltaz)

In [ ]:
print dac.keys()

In [ ]:
d1 = Table.read('/Users/hoyleb/DATA/DES/PHOTOZ/Y1v103Spec_notin_Alhambra.fits')
PRIMUS = np.array([i.replace(' ','')== 'PRIMUS' for i in d1['SOURCE']])
i = np.array(d1['FLAG'] != 3) * np.array(PRIMUS == False)

In [ ]:
d1[i].write('/Users/hoyleb/DATA/DES/PHOTOZ/Y1v103Spec_notin_Alhambra_noPrimus_flagne3.fits')

In [ ]:
print  np.sum(np.array(PRIMUS == False)), np.sum(np.array(d1['FLAG'] != 3) )

In [ ]:
len(d1), np.sum(i)
d1 = d1[i]

In [ ]:
import sml_mla as ml
import calculate_weights as cw

test_d = '/Users/hoyleb/DATA/DES/PHOTOZ/merged.Y1A1_GOLD_1_0_3_000001.fits_alhambra.fits'
weight_f = ['MAG_AUTO_G','MAG_AUTO_R','MAG_AUTO_I','MAG_AUTO_Z','MAG_AUTO_I-MAG_AUTO_R',
            'MAG_AUTO_G-MAG_AUTO_R','MAG_AUTO_I-MAG_AUTO_Z']

inptestW, test_info = ml.dataSet(test_d, weight_f, ['COADD_OBJECTS_ID', 'zb_1']).loadData()
inpTrain, train_info = ml.dataSet(d1, weight_f, ['COADD_OBJECTS_ID', 'SPEC_Z']).loadData()


In [ ]:

n_neighbors_array = [1,2,3,4,5,10]
train_test = cw.best_lima_knn_weights(
        inpTrain,
        inptestW,
        verbose=1,
        n_neighbors_array=n_neighbors_array
    )

In [ ]:
d1['WEIGHTS_TO_ALHAMBRA'] = train_test

In [ ]:
zbins = np.linspace(0, 1.6, num = 100)
train_t_z = gss_kde(np.array(train_info['SPEC_Z']), weights=train_test).evaluate(zbins) 
train_t_nw_z = gss_kde(np.array(train_info['SPEC_Z'])).evaluate(zbins) 
test_t_z = gss_kde(np.array(test_info['zb_1'])).evaluate(zbins) 

In [ ]:
plt.plot(zbins, train_t_nw_z, label='UnWeighted spec-z dist')
plt.plot(zbins, train_t_z, label='Weighted spec-z dist')
plt.plot(zbins, test_t_z, label='ALHAMBRA photo-z dist')
plt.legend()
plt.xlabel('redshift')
plt.ylabel('density')

In [ ]:
for i, mag in enumerate(weight_f):
    mbins = np.linspace(np.amin(inptestW[:, i]),np.amax(inptestW[:, i]), num = 100)
    train_t_z = gss_kde(np.array(inpTrain[:, i]), weights=train_test).evaluate(mbins) 
    test_t_z = gss_kde(np.array(inptestW[:, i])).evaluate(mbins) 
    f = plt.figure()
    plt.plot(zbins, train_t_z, label='Weighted spec dist')
    plt.plot(zbins, test_t_z, label='ALHAMBRA dist')
    plt.legend()
    plt.xlabel(mag)
    plt.ylabel('density')

In [ ]:
i = np.array(d1['WEIGHTS_TO_ALHAMBRA'] > 0)
print np.sum(i)
_ = plt.hist(d1['WEIGHTS_TO_ALHAMBRA'][i],bins=100)

In [ ]:
d2 = d1[i]

In [ ]:
d2.write('/Users/hoyleb/DATA/DES/PHOTOZ/Y1v103Spec_notin_Alhambra_noPrimus_flagne3.weights.fits')

In [ ]:
#make predictions on alhambra using 
#%>des_ml_output.py ml_alhambra.outxv20.p merged.Y1A1_GOLD_1_0_3_000001.fits_alhambra.fits MAGERR_AUTO_I,MAGERR_AUTO_R,MAGERR_AUTO_Z,MAGERR_AUTO_G,MAG_AUTO_G,MAG_AUTO_R,MAG_AUTO_I,MAG_AUTO_Z,COADD_OBJECTS_ID,zb_1
res = Table.read('/Users/hoyleb/DATA/DES/PHOTOZ/merged.Y1A1_GOLD_1_0_3_000001.fits_alhambra.fitsml_alhambra.outxv20.DES.point.predictions.fits')

In [ ]:
print res.keys()
i = np.array(res['MAGERR_AUTO_I']  > 0 ) * np.array(res['MAGERR_AUTO_I'] < 0.2) * np.array(
    res['MAG_AUTO_I'] - res['MAG_AUTO_Z'] > -10) * np.array(res['MAG_AUTO_Z'] > 16) * np.array(
    res['MAG_AUTO_Z']<25) * np.array(res['MAG_AUTO_I'] > 16) * np.array(res['MAG_AUTO_I']<23.5) * np.array(
    res['MAG_AUTO_R'] > 16) * np.array(res['MAG_AUTO_R']<25) * np.array(
    res['MAG_AUTO_G'] > 16) * np.array(res['MAG_AUTO_G']<25)
res = res[i]

In [ ]:

plt.plot(res['zb_1'], res['Z'],',')
plt.xlabel('Alhambra-z')
plt.ylabel('ML-z')
plt.plot([0, 3], [0, 3], '--', linewidth=2, color=almost_black)
plt.xlim(0, 3)
plt.ylim(0, 3)

deltaz = ml_m.DeltaZ(res['zb_1'], res['Z'])
print 'stats on (ALH-z - ML-z)/(1 + ALH-z)'
print ml_m.getStats(deltaz)

zbins = np.array([0, 0.3, 0.5, 0.7, 0.9, 1.0, 1.3])
for i in range(len(zbins) - 1):
    ind = (res['Z'] < zbins[i+1]) * (res['Z'] >= zbins[i])
    print '{:0.2f} < z <{:0.2f}'.format(zbins[i], zbins[i + 1])
    sts =  ml_m.getStats(deltaz[ind])
    print 'median: {median:0.3f} sigma_68  {sigma_68:0.3f} , outfrac  {outlierFrac:0.3f}'.format(**sts)



In [ ]:
_ = plt.hist(res['MAGERR_AUTO_I'], bins=50)

In [ ]:
inp_res, res_info = ml.dataSet(res, weight_f, ['COADD_OBJECTS_ID']).loadData()
for i, mag in enumerate(weight_f):
    f = plt.figure()
    plt.plot(inp_res[: ,i], deltaz, ',', label='Delta_z')
    plt.legend()
    plt.xlabel(mag)
    plt.ylabel('Delta_z')


In [ ]:
#kmeans clusters on 
clf = km.fit_kmeans(inp_res, 25, verbose=True)
clusters = km.get_clusters(clf, inp_res)
keep = np.zeros(len(clusters), dtype=bool)
deltaz = ml_m.DeltaZ(res['zb_1'], res['Z'])
for i in np.unique(clusters):
    ind = clusters == i
    indx = np.arange(len(ind))[ind]
    np.random.shuffle(indx)
    
    if np.sum(ind) > 75:
        sts =  ml_m.getStats(deltaz[indx[0: int(len(indx)*0.3)]])
        print 'cluster {:}'.format(i)
        print 'median: {median:0.3f} sigma_68  {sigma_68:0.3f} , outfrac  {outlierFrac:0.3f}'.format(**sts)
        if sts['sigma_68'] < 0.1:
            keep[indx[int(len(indx) * 0.3): ]] = True
        

In [ ]:
np.sum(keep), len(keep)

In [ ]:
zbins = np.array([0, 0.3, 0.5, 0.7, 0.9, 1.0, 1.3])
res_keep = res[keep]

plt.plot(res_keep['zb_1'], res_keep['Z'],',')
plt.xlabel('Alhambra-z')
plt.ylabel('ML-z')
plt.plot([0, 3], [0, 3], '--', linewidth=2, color=almost_black)
plt.xlim(0, 3)
plt.ylim(0, 3)

deltaz_keep =  ml_m.DeltaZ(res_keep['zb_1'], res_keep['Z'])
print 'stats on (ALH-z - alh-z)/(1 + ALH-z)' ,len(res_keep), len(res) 
print ml_m.getStats(deltaz_keep)
for i in range(len(zbins) - 1):
    ind = (res_keep['Z'] < zbins[i+1]) * (res_keep['Z'] >= zbins[i])
    ind1 = (res['Z'] < zbins[i+1]) * (res['Z'] >= zbins[i])
    
    print '{:0.2f} < z <{:0.2f}'.format(zbins[i], zbins[i + 1])
    sts =  ml_m.getStats(deltaz_keep[ind])
    print 'median: {median:0.3f} sigma_68  {sigma_68:0.3f} , outfrac  {outlierFrac:0.3f}'.format(**sts), 'Culled data {:0.2f}%'.format(100 - np.sum(ind)*100.0 / np.sum(ind1))


In [ ]:
dspec = Table.read('/Users/hoyleb/DATA/DES/PHOTOZ/Y1v103Spec_notin_Alhambra.fits')

In [ ]:
dspec = dspec[dspec['FLAG'] != 3]
print np.unique(np.array(dspec['SOURCE']))
SOURCE = np.array([i.replace(' ', '') for i in dspec['SOURCE']])

In [ ]:
zbins = np.linspace(0, 1.6, num = 100)
test_t_z = gss_kde(np.array(test_info['zb_1'])).evaluate(zbins) 
best_chi2 = 1e99
for i in range(500):
    sample = np.random.choice(np.unique(SOURCE), size=np.random.randint(1, len(np.unique(SOURCE))), replace=False )
    ind = np.zeros(len(SOURCE), dtype=bool)
    for j in sample:
        ind += j == SOURCE 
    train_t_nw_z = gss_kde(np.array(dspec['SPEC_Z'][ind])).evaluate(zbins) 
    
    chi2 = np.sum((train_t_nw_z - test_t_z)**2)
    if chi2 < best_chi2:
        best_chi2 = chi2
        best_sample = sample
        print chi2, sample

ind = np.zeros(len(SOURCE), dtype=bool)
for j in best_sample:
    ind += j == SOURCE 
train_t_nw_z = gss_kde(np.array(dspec['SPEC_Z'][ind])).evaluate(zbins) 

plt.plot(zbins, train_t_nw_z, label='Selected spec-z dist')
plt.plot(zbins, test_t_z, label='ALHAMBRA photo-z dist')
plt.legend()
plt.xlabel('redshift')
plt.ylabel('density')
plt.legend()
plt.xlabel('redshift')
plt.ylabel('density')




In [ ]:
best_sample = ['EBOSS_DES_ELG', 'VUDS', 'DEEP2', 'SDSS_OZDES' ,'DES_AAOMEGA' ,'DR12_LOWZ' ,'SPARCS', 'ATLAS']
best_sample = ['3DHST', 'PANSTARRS']

ind = np.zeros(len(SOURCE), dtype=bool)
for j in best_sample:
    ind += j == SOURCE 
    
print np.sum(ind)
train_t_nw_z = gss_kde(np.array(dspec['SPEC_Z'][ind])).evaluate(zbins) 

plt.plot(zbins, train_t_nw_z, label='Selected spec-z dist')
plt.plot(zbins, test_t_z, label='ALHAMBRA photo-z dist')
plt.legend()
plt.xlabel('redshift')
plt.ylabel('density')
plt.legend()
plt.xlabel('redshift')
plt.ylabel('density')

In [ ]:
print np.sum(ind)

test_d = '/Users/hoyleb/DATA/DES/PHOTOZ/merged.Y1A1_GOLD_1_0_3_000001.fits_alhambra.fits'
weight_f = ['MAG_AUTO_G','MAG_AUTO_R','MAG_AUTO_I','MAG_AUTO_Z','MAG_AUTO_I-MAG_AUTO_R',
            'MAG_AUTO_G-MAG_AUTO_R','MAG_AUTO_I-MAG_AUTO_Z']
inptestW, test_info = ml.dataSet(res, weight_f, ['COADD_OBJECTS_ID', 'zb_1']).loadData()
inpTrain, train_info = ml.dataSet(dspec[ind], weight_f, ['COADD_OBJECTS_ID', 'SPEC_Z']).loadData()


In [ ]:
n_neighbors_array = [1,2,3,4,5,10]
train_test = cw.best_lima_knn_weights(
        inpTrain,
        inptestW,
        verbose=1,
        n_neighbors_array=n_neighbors_array
    )

In [ ]:
dspec = dspec[ind]
dspec['WEIGHTS_TO_ALHAMBRA'] = train_test

In [ ]:
train_t_nw_z = gss_kde(np.array(dspec['SPEC_Z'])).evaluate(zbins) 
zbins = np.linspace(0, 1.6, num = 100)
train_t_z = gss_kde(np.array(dspec['SPEC_Z']), weights=train_test).evaluate(zbins) 
train_t_nw_z = gss_kde(np.array(dspec['SPEC_Z'])).evaluate(zbins) 
test_t_z = gss_kde(np.array(test_info['zb_1'])).evaluate(zbins) 
plt.plot(zbins, train_t_nw_z, label='UnWeighted spec-z dist')
plt.plot(zbins, train_t_z, label='Weighted spec-z dist')
plt.plot(zbins, test_t_z, label='ALHAMBRA photo-z dist')
plt.legend()
plt.xlabel('redshift')
plt.ylabel('density')


In [ ]:
dspec.write('/Users/hoyleb/DATA/DES/PHOTOZ/Y1v103Spec_notin_Alhambra.selected_sample1.fits')

In [ ]:
#CHFTLS 
res_chf = Table.read('/Users/hoyleb/DATA/DES/PHOTOZ/Y1A1_GOLD_1_0_3_COSMOS2015_matched.fitsml_alhambra.outxv20.DES.point.predictions.fits')

In [ ]:
keep = np.ones(len(res_chf), dtype=bool)
keep = np.array(res_chf['MAG_AUTO_I'] < 24) * np.array(res_chf['MAGERR_AUTO_I'] < 0.4) *np.array(res_chf['MAG_AUTO_I'] > 17)
zbins = np.array([0, 0.3, 0.5, 0.7, 0.9, 1.0, 1.3])
res_keep = res_chf[keep]

plt.plot(res_keep['ZMINCHI2'], res_keep['Z'],',')
plt.xlabel('Cosmos-z')
plt.ylabel('ML-z')
plt.plot([0, 3], [0, 3], '--', linewidth=2, color=almost_black)
plt.xlim(0, 3)
plt.ylim(0, 3)

deltaz_keep =  ml_m.DeltaZ(res_keep['ZMINCHI2'], res_keep['Z'])
print 'stats on (COSMOS-z - alh-z)/(1 + COSMOS-z)' ,len(res_keep), len(res) 
print ml_m.getStats(deltaz_keep)
for i in range(len(zbins) - 1):
    ind = (res_keep['Z'] < zbins[i+1]) * (res_keep['Z'] >= zbins[i])
    ind1 = (res_chf['Z'] < zbins[i+1]) * (res_chf['Z'] >= zbins[i])
    print '{:0.2f} < z <{:0.2f}'.format(zbins[i], zbins[i + 1])
    sts =  ml_m.getStats(deltaz_keep[ind])
    print 'median: {median:0.3f} sigma_68  {sigma_68:0.3f} , outfrac  {outlierFrac:0.3f}'.format(**sts), 'Culled data {:0.2f}%'.format(100 - np.sum(ind)*100.0 / np.sum(ind1))


In [ ]:
#kmeans clusters on 
inp_res, res_info = ml.dataSet(res_chf, weight_f, ['COADD_OBJECTS_ID']).loadData()

clf = km.fit_kmeans(inp_res, 50, verbose=True)
clusters = km.get_clusters(clf, inp_res)
deltaz =  ml_m.DeltaZ(res_chf['ZMINCHI2'], res_chf['Z'])
keep = np.zeros(len(clusters), dtype=bool)
keep_cluster = []
for i in np.unique(clusters):
    ind = clusters == i
    indx = np.arange(len(ind))[ind]
    np.random.shuffle(indx)
    
    if np.sum(ind) > 75:
        sts =  ml_m.getStats(deltaz[indx[0: int(len(indx)*0.3)]])
        print 'cluster {:}'.format(i)
        print 'median: {median:0.3f} sigma_68  {sigma_68:0.3f} , outfrac  {outlierFrac:0.3f}'.format(**sts)
        if sts['sigma_68'] < 0.1:
            keep[indx[int(len(indx) * 0.3): ]] = True
            keep_cluster.append(i)
print keep_cluster

In [ ]:
zbins = np.array([0, 0.3, 0.5, 0.7, 0.9, 1.0, 1.3])
res_keep = res_chf[keep]

plt.plot(res_keep['ZMINCHI2'], res_keep['Z'],',')
plt.xlabel('Cosmos-z')
plt.ylabel('ML-z')
plt.plot([0, 3], [0, 3], '--', linewidth=2, color=almost_black)
plt.xlim(0, 3)
plt.ylim(0, 3)

deltaz_keep =  ml_m.DeltaZ(res_keep['ZMINCHI2'], res_keep['Z'])
print 'stats on (Cosmos-z - alh-z)/(1 + ALH-z)' ,len(res_keep), len(res_chf) 
print ml_m.getStats(deltaz_keep)
for i in range(len(zbins) - 1):
    ind = (res_keep['Z'] < zbins[i+1]) * (res_keep['Z'] >= zbins[i])
    ind1 = (res_chf['Z'] < zbins[i+1]) * (res_chf['Z'] >= zbins[i])
    
    print '{:0.2f} < z <{:0.2f}'.format(zbins[i], zbins[i + 1])
    sts =  ml_m.getStats(deltaz_keep[ind])
    print 'median: {median:0.3f} sigma_68  {sigma_68:0.3f} , outfrac  {outlierFrac:0.3f}'.format(
        **sts), 'Culled data {:0.2f}%'.format(100 - np.sum(ind) * (100.0 / 0.66) / np.sum(ind1))


In [ ]:
#kmeans clusters on 
inp_wl, res_wl = ml.dataSet('/Users/hoyleb/DATA/DES/PHOTOZ/WL_INFO_0_1M_ID.Y1A1_GOLD_1_0_3.nside128.fits', 
                          weight_f, ['COADD_OBJECTS_ID']).loadData()
#how much WL data do we throw away?
clusters_wl = km.get_clusters(clf, inp_wl)


In [ ]:
#kmeans clusters on 
print keep_cluster
kept_wl = 0
indkeep = np.zeros(len(clusters_wl), dtype=bool)
for i in keep_cluster:
    ind = i == clusters_wl
    if np.sum(ind) > 0:
        kept_wl += np.sum(ind) 
        indkeep[ind] = True
print 'WL sample kept={:}, total={:} fraction kept: {:} %'.format(kept_wl , len(clusters_wl), kept_wl * 100.0 / len(clusters_wl))

In [ ]:
for i, mag in enumerate(weight_f):
    mbins = np.linspace(np.amin(inp_wl[indkeep, i]),np.amax(inp_wl[indkeep, i]), num = 100)
    uncut_m = gss_kde(np.array(inp_wl[:, i])).evaluate(mbins) 
    cut_m = gss_kde(np.array(inp_wl[indkeep, i])).evaluate(mbins) 
    f = plt.figure()
    plt.plot(mbins, uncut_m, label='Full WL dist')
    plt.plot(mbins, cut_m, label='Culled WL dist')
    plt.legend()
    plt.xlabel(mag)
    plt.ylabel('density')


In [ ]:
d=Table.read('/Users/hoyleb/DATA/ALHAMBRA/alhambra_survey_photoZ_catalogue_2013.fits')
print len(d)
d = d[d['F814W'] < 24]
d = d[d['Stellar_Flag'] <=0.7 ]
print len(d)

In [ ]:
_ = plt.hist(d['Stellar_Flag'])

f = plt.figure()

for i in np.unique(d['Field']):
    ind = np.array(i == d['Field'])
    plt.plot(d['RA'][ind], d['Dec'][ind], '--', label=str(i))

plt.legend()
plt.xlabel('RA')
plt.ylabel('Dec')
#plt.ylim(0,10)
#plt.xlim(30,170)

ind = np.array(4 == d['Field'])
d[ind].write('/Users/hoyleb/DATA/ALHAMBRA/alhambra_survey_photoZ_catalogue_2013.DESY1A1.fits')
d[ind == False].write('/Users/hoyleb/DATA/ALHAMBRA/alhambra_survey_photoZ_catalogue_2013.nonDESY1A1.fits')

In [ ]:
dn = Table.read('/Users/hoyleb/DATA/ALHAMBRA/alhambra_survey_photoZ_catalogue_2013.nonDESY1A1.fitsphotoz.DesY1A1.xv100.DES.point.predictions.fits')
#comparison with kfold cross validation:
res = p.load(open('/Users/hoyleb/DATA/ALHAMBRA/photoz.DesY1A1.xv100.p', 'r'))
scoreXv = np.zeros(0)
truths = np.zeros(0)
indexXv = np.zeros(0, dtype=int)
prediction = np.zeros(0)
ID = np.zeros(0, dtype=int)

for ii, j in enumerate(res['results']['resXv']):
    scoreXv = np.append(scoreXv, j['scoreXv'])
    prediction = np.append(prediction, j['predict'])
    indexXv = np.append(indexXv, j['index'])
    truths = np.append(truths, j['outputs'])
    ID = np.append(ID, j['id'])


In [ ]:
zbins = np.array([0, 0.3, 0.5, 0.7, 0.9, 1.0, 1.3])

deltaz_keep =  ml_m.DeltaZ(truths, prediction)
print 'stats on (AH-z - ML-z)/(1 + ALH-z)'
print ml_m.getStats(deltaz_keep)
for i in range(len(zbins) - 1):
    ind = (prediction < zbins[i+1]) * (prediction >= zbins[i])
    print '{:0.2f} < z <{:0.2f}'.format(zbins[i], zbins[i + 1])
    sts =  ml_m.getStats(deltaz_keep[ind])
    print 'median: {median:0.3f} sigma_68  {sigma_68:0.3f} , outfrac  {outlierFrac:0.3f}'.format(**sts)


<h3>Make COSMOS data clean</h3>



In [ ]:
d = Table.read('/Users/hoyleb/DATA/DES/PHOTOZ/Y1A1_GOLD_1_0_3_COSMOS2015_matched.fits')
print len(d)
d = d[d['MAG_AUTO_I']<23.5]
d = d[d['MAG_AUTO_I']>15]
d = d[d['MAGERR_AUTO_I']>0]
d = d[d['MAGERR_AUTO_I']<0.3]

for i in ['MAG_AUTO_R', 'MAG_AUTO_G', 'MAG_AUTO_I', 'MAG_AUTO_Z']:
    d = d[d[i]<27]
    d = d[d[i]>12]

    
for i in ['MAGERR_AUTO_R', 'MAGERR_AUTO_G', 'MAGERR_AUTO_I', 'MAGERR_AUTO_Z']:
    d = d[d[i]<0.5]
    d = d[d[i]>0]

    
print len(d)


In [ ]:
d.write('/Users/hoyleb/DATA/DES/PHOTOZ/Y1A1_GOLD_1_0_3_COSMOS2015_matched.cleanmags.fits')

<h3>Train on DES mags in alhambra field 4</h3>
Do we hit requirements, assuming no cosmic variance

In [ ]:
res = p.load(open('/Users/hoyleb/DATA/ALHAMBRA/photoz.Alhabra.Desmags.z.xv50.p', 'r'))
scoreXv = np.zeros(0)
truths = np.zeros(0)
indexXv = np.zeros(0, dtype=int)
prediction = np.zeros(0)
ID = np.zeros(0, dtype=int)

for ii, j in enumerate(res['results']['resXv']):
    scoreXv = np.append(scoreXv, j['scoreXv'])
    prediction = np.append(prediction, j['predict'])
    indexXv = np.append(indexXv, j['index'])
    truths = np.append(truths, j['outputs'])
    ID = np.append(ID, j['id'])

'Print DES g/r/i/z photometry'
zbins = np.array([0, 0.3, 0.5, 0.7, 0.9, 1.0, 1.3])

deltaz_keep =  ml_m.DeltaZ(truths, prediction)
print 'stats on (AH-z - ML-z)/(1 + ALH-z)'
print ml_m.getStats(deltaz_keep)
for i in range(len(zbins) - 1):
    ind = (prediction < zbins[i+1]) * (prediction >= zbins[i])
    print '{:0.2f} < z <{:0.2f}'.format(zbins[i], zbins[i + 1])
    sts =  ml_m.getStats(deltaz_keep[ind])
    print 'median: {median:0.3f} sigma_68  {sigma_68:0.3f} , outfrac  {outlierFrac:0.3f}'.format(**sts)

    

Do we hit requirements when we use pseudo DES photometry from other fields?

In [ ]:
d = Table.read('/Users/hoyleb/DATA/ALHAMBRA/alhambra_survey_photoZ_catalogue_2013.fits.pseudoDesMags.fitsphotoz.Alhabra.Desmags.z.xv50.DES.point.predictions.fits')
d = d[d['Stellar_Flag'] < 0.7]
d = d[d['F814W'] < 24]
d = d[d['MAG_AUTO_I']<23.5]
print 'Pseudo des photometery'

print 'All fields together'

truths = d['zb_1']
prediction = d['Z']
deltaz_keep =  ml_m.DeltaZ(truths, prediction)
print 'stats on (AH-z - ML-z)/(1 + ALH-z)'
print ml_m.getStats(deltaz_keep)
for i in range(len(zbins) - 1):
    ind = (prediction < zbins[i+1]) * (prediction >= zbins[i])
    print '{:0.2f} < z <{:0.2f}'.format(zbins[i], zbins[i + 1])
    sts =  ml_m.getStats(deltaz_keep[ind])
    print 'median: {median:0.3f} sigma_68  {sigma_68:0.3f} , outfrac  {outlierFrac:0.3f}'.format(**sts)

    print 'wl_val_test', wl_val_test(np.mean(prediction[ind]), np.mean(truths[ind]))
    
for field in np.unique(d['Field']):
    ind  = d['Field'] == field
    print ' '
    print 'Field: ',field
    
    deltaz_keep =  ml_m.DeltaZ(truths[ind], prediction[ind])
    print 'stats on (AH-z - ML-z)/(1 + ALH-z)'
    print ml_m.getStats(deltaz_keep)
    for i in range(len(zbins) - 1):
        ind1 = (prediction[ind] < zbins[i+1]) * (prediction[ind] >= zbins[i])
        print '{:0.2f} < z <{:0.2f}'.format(zbins[i], zbins[i + 1])
        sts =  ml_m.getStats(deltaz_keep[ind1])
        print 'median: {median:0.3f} sigma_68  {sigma_68:0.3f} , outfrac  {outlierFrac:0.3f}'.format(**sts)
        print 'wl_val_test', wl_val_test(np.mean(prediction[ind][ind1]), np.mean(truths[ind][ind1]))
    

    


In [ ]:
zbins = np.array([0, 0.3, 0.5, 0.7, 0.9, 1.0, 1.3])

for i in np.unique(d['Field']):
    ind = np.array(i == d['Field'])
    #plt.plot(d['Z'][ind], d['zb_1'][ind], 'o', label='Region:{:} Number {:}'.format(str(i), np.sum(ind)))

    d_ = d[ind]
    deltaz_keep =  ml_m.DeltaZ(d_['zb_1'], d_['Z'])
    hb = np.histogram(deltaz_keep, bins=np.arange(150)/75.0 - 1)
    plt.plot(hb[1][1:] - 1.0/150, hb[0] *1.0 / np.sum(hb[0]), label='Region:{:} Number {:}'.format(str(i), np.sum(ind)))
    print 'stats on (AH-z - ML-z)/(1 + ALH-z)'
    print ml_m.getStats(deltaz_keep)
    for i in range(len(zbins) - 1):
        ind = (d_['Z'] < zbins[i+1]) * (d_['Z'] >= zbins[i])
        print '{:0.2f} < z <{:0.2f}'.format(zbins[i], zbins[i + 1])
        sts =  ml_m.getStats(deltaz_keep[ind])
        print 'median: {median:0.3f} sigma_68  {sigma_68:0.3f} , outfrac  {outlierFrac:0.3f}'.format(**sts)
        
plt.legend()
plt.xlabel('ALH_z - ML_z')
plt.ylabel('#')
plt.xlim(-0.2,0.2)

f = plt.figure()
#now make pdf plots
zbn = np.linspace(0,3,300)
col = ['red', 'blue', 'green', 'purple', 'grey', 'orange','red']
cnt=0
for i in np.unique(d['Field']):
    f = plt.figure()
    ind = np.array(i == d['Field'])
    #plt.plot(d['Z'][ind], d['zb_1'][ind], 'o', label='Region:{:} Number {:}'.format(str(i), np.sum(ind)))
    d_ = d[ind]
    true_z =  gss_kde(d_['zb_1']).evaluate(zbn) 
    pred_z = gss_kde(d_['Z']).evaluate(zbn) 
   
    plt.plot(zbn, true_z,'-', color=col[cnt], label='Alh-z region:{:}'.format(str(i)))
    plt.plot(zbn, pred_z,'--', color=col[cnt], label='ML-z region:{:}'.format(str(i)))
    cnt +=1
    plt.legend()
    plt.xlabel('Redshift')
    plt.ylabel('#')


true_z =  gss_kde(truths).evaluate(zbn) 
pred_z = gss_kde(prediction).evaluate(zbn) 
f = plt.figure()
plt.plot(zbn, true_z,'-', color=col[cnt], label='Alh-z COSMOS')
plt.plot(zbn, pred_z,'--', color=col[cnt], label='ML-z COSMOS')
cnt +=1
plt.legend()
plt.xlabel('Redshift')
plt.ylabel('#')


In [ ]:
#now apply weights to alhambra data and see how the dndz looks for different fields, after weighting is applied.
d1= Table.read('/Users/hoyleb/DATA/ALHAMBRA/alhambra_survey_photoZ_catalogue_2013.DESY1A1.fits')
dn = Table.read('/Users/hoyleb/DATA/ALHAMBRA/alhambra_survey_photoZ_catalogue_2013.nonDESY1A1.fits')

weight_f = ['F365W', 'dF365W', 'F396W', 'dF396W', 'F427W', 'dF427W', 'F458W', 'dF458W', 'F489W', 'dF489W', 'F520W', 
            'dF520W', 'F551W', 'dF551W', 'F582W', 'dF582W', 'F613W', 'dF613W', 'F644W', 'dF644W', 'F675W', 'dF675W', 
            'F706W', 'dF706W', 'F737W', 'dF737W', 'F768W', 'dF768W', 'F799W', 'dF799W', 'F830W', 'dF830W', 'F861W', 
            'dF861W', 'F892W', 'dF892W', 'F923W', 'dF923W', 'F954W', 'dF954W', 'J', 'dJ', 'H', 'dH', 'KS', 'dKS', 
            'F814W', 'dF814W']

inpTrain, train_info = ml.dataSet(d1, weight_f, ['ID', 'zb_1']).loadData()
inptestW, test_info = ml.dataSet(dn, weight_f, ['ID', 'Field', 'zb_1']).loadData()

zbins = np.linspace(0, 1.6, num = 100)
train_t_nw_z = gss_kde(np.array(train_info['zb_1'])).evaluate(zbins) 


n_neighbors_array = [5, 10, 20, 50]
weights_alahmbra = {'features': weight_f, 'Field_weights': {}}

for i in np.unique(test_info['Field']):
    ind_test = test_info['Field'] == i
    train_test = cw.best_lima_knn_weights(
            inpTrain,
            inptestW[ind_test],
            verbose=1,
            n_neighbors_array=n_neighbors_array
        )
    
    weights_alahmbra['Field_weights'][i] = train_test
    
    train_t_z = gss_kde(np.array(train_info['zb_1']), weights=train_test).evaluate(zbins) 
    test_t_z = gss_kde(np.array(test_info['zb_1'][ind_test])).evaluate(zbins) 
    f = plt.figure()
    plt.plot(zbins, train_t_nw_z, label='Train alh-z unweighted')
    plt.plot(zbins, train_t_z, label='Train alh-z weighted')
    plt.plot(zbins, test_t_z, label='Test alh-z')
    plt.legend()
    plt.xlabel('redshift')
    plt.ylabel('density')


In [ ]:
#train a photo machine to make predictions using these weights
alh_clf = {}
for i in np.unique(test_info['Field']):
    ind_test = test_info['Field'] == i
    train_test = weights_alahmbra['Field_weights'][i]
    
    results, score = k_fold_val(inpTrain, train_info['zb_1'], train_test, 10, 30)
    alh_clf[i] = {}
    alh_clf[i]['clf'] = results['clf']
    alh_clf[i]['results'] = results
    alh_clf[i]['predict_test'] = results['clf'].predict(inptestW[ind_test])
    alh_clf[i]['truth_test'] = test_info['zb_1'][ind_test]
    

In [ ]:
def zarr_from_ml(z_, sigma_, weights_):
    weights1 = normalise(weights_)
    ind = np.random.choice(np.arange(len(z_), dtype=int), size=np.ceil(1.0/np.sum(weights1**2)), p=weights1, replace=True)
    zrr = z_[ind] + np.random.normal(size=len(ind)) * sigma_[ind]
    cnt =0
    while np.sum(zrr<0) > 0:
        ind1 = zrr <0
        zrr[ind1] = z_[ind][ind1]+ np.random.normal(size=np.sum(ind1)) * sigma_[ind][ind1]
    return zrr

def wl_val_test(x1,x2):
    #if np.abs(x1-x2)<0.02:
    #    return True
    return np.abs(x1-x2)


In [ ]:
tom_zbins = [0, 0.3, 0.5 , 0.7, 0.9, 1.1, 1.3]
col = ['red', 'blue', 'green', 'purple', 'grey', 'orange','red']
for i in np.unique(test_info['Field']):
    print 'prediction for field ',i
    ind_test = test_info['Field'] == i
    sigma, z_mc = allPredictions(alh_clf[i]['clf'], inptestW[ind_test])
    
    f = plt.figure()
    for j in range(len(tom_zbins)-1):
        ind = (alh_clf[i]['predict_test'] <= tom_zbins[j+1]) * (alh_clf[i]['predict_test'] > tom_zbins[j])
        z_txt= '{:0.2f}<z<{:0.2f}'.format(tom_zbins[j], tom_zbins[j+1])
        print z_txt
        pz_ = alh_clf[i]['predict_test'][ind] + np.random.normal(size=np.sum(ind))*sigma[ind]
        wlres =  wl_val_test(np.mean(alh_clf[i]['truth_test'][ind]), np.mean(pz_))
        print wlres
      
        
        #pz_ = z_mc[ind] 
        train_t_z = gss_kde(pz_).evaluate(zbins) 
        test_t_z = gss_kde(alh_clf[i]['truth_test'][ind]).evaluate(zbins) 
    
        plt.plot(zbins, train_t_z, '--',color=col[j], alpha=0.8, linewidth=3, label='ML-z: {:0.2f} '.format(wlres) )
        plt.plot(zbins, test_t_z, '-',color=col[j], alpha=0.8, linewidth=3,label='Alh-z: '+ z_txt)
    plt.legend()
    plt.xlabel('redshift')
    plt.ylabel('density')
    plt.title('Field {:}'.format(i))

In [ ]:
tom_zbins = [0, 1.3]
col = ['red', 'blue', 'green', 'purple', 'grey', 'orange','red']
for i in np.unique(test_info['Field']):
    print 'prediction for field ',i
    ind_test = test_info['Field'] == i
    sigma, z_mc = allPredictions(alh_clf[i]['clf'], inptestW[ind_test])
    
    
    f = plt.figure()
    for j in range(len(tom_zbins)-1):
        ind = (alh_clf[i]['predict_test'] <= tom_zbins[j+1]) * (alh_clf[i]['predict_test'] > tom_zbins[j])
        z_txt= '{:0.2f}<z<{:0.2f}'.format(tom_zbins[j], tom_zbins[j+1])
        print z_txt
        wlres =  wl_val_test(np.mean(alh_clf[i]['truth_test'][ind]), np.mean(alh_clf[i]['predict_test'][ind]))
        print wlres
      
        pz_ = alh_clf[i]['predict_test'][ind]# + np.random.normal(size=np.sum(ind))*sigma[ind]
        pz_ = z_mc[ind] 
        train_t_z = gss_kde(pz_).evaluate(zbins) 
        test_t_z = gss_kde(alh_clf[i]['truth_test'][ind]).evaluate(zbins) 
    
        plt.plot(zbins, train_t_z, '--',color=col[j], alpha=0.8, linewidth=3, label='ML-z: {:0.2f} '.format(wlres))
        plt.plot(zbins, test_t_z, '-',color=col[j], alpha=0.8, linewidth=3,label='Alh-z: ' + z_txt)
    plt.legend()
    plt.xlabel('redshift')
    plt.ylabel('density')
    plt.title('Field {:}'.format(i))
    
   


In [ ]:
#ignore weighting and calcualte metrics
res = p.load(open('/Users/hoyleb/DATA/ALHAMBRA/photoz.DesY1A1.xv100.p', 'r'))
inptestW_, test_info_ = ml.dataSet(dn, res['features']['input'], ['ID', 'Field', 'zb_1']).loadData()
pz_ = res['clf'].predict(inptestW_)
sigma, z_mc = allPredictions(res['clf'], inptestW_)


In [ ]:
tom_zbins = [0, 0.3, 0.5, 0.7, 0.9, 1.1, 1.3]
col = ['red', 'blue', 'green', 'purple', 'grey', 'orange','red']
for i in np.unique(test_info_['Field']):
    print 'prediction for field ',i
    ind_test = test_info['Field'] == i
    
    f = plt.figure()
    for j in range(len(tom_zbins)-1):
        ind = (pz_[ind_test] <= tom_zbins[j+1]) * (pz_[ind_test] > tom_zbins[j])
        z_txt= '{:0.2f}<z<{:0.2f}'.format(tom_zbins[j], tom_zbins[j+1])
        print z_txt
        wlres =  wl_val_test(np.mean(test_info_['zb_1'][ind_test][ind]), np.mean(pz_[ind_test][ind]))
        print wlres
      
        pz = pz_[ind_test][ind] #+ np.random.normal(size=np.sum(ind))*sigma[ind_test][ind]
        pz_ = z_mc[ind] 
        train_t_z = gss_kde(pz).evaluate(zbins) 
        test_t_z = gss_kde(test_info_['zb_1'][ind_test][ind]).evaluate(zbins) 
        
        plt.plot(zbins, train_t_z, '--',color=col[j], alpha=0.8, linewidth=3, label='ML-z: {:0.2f} '.format(wlres) + z_txt)
        plt.plot(zbins, test_t_z, '-',color=col[j], alpha=0.8, linewidth=3,label='Alh-z: ' + z_txt)
    plt.legend()
    plt.xlabel('redshift')
    plt.ylabel('density')
    plt.title('Field {:}'.format(i))
    

<h3>Train on mutli-fields DES pseudo photometry</h3>

make predictions on some held out fields, and also field 4

In [ ]:
d = Table.read('/Users/hoyleb/DATA/ALHAMBRA/alhambra_survey_photoZ_catalogue_2013.fits.pseudoDesMags.fits')
print len(d)
d = d[d['Stellar_Flag'] < 0.7]
d = d[d['F814W'] < 24]
print len(d)

d2 = Table.read('/Users/hoyleb/DATA/DES/PHOTOZ/merged.Y1A1_GOLD_1_0_3_000001.fits_alhambra.fits')
print len(d2)
d2 = d2[d2['Stellar_Flag'] < 0.7]
d2 = d2[d2['F814W'] < 24]
print len(d2)

feats_des = ['MAG_AUTO_G','MAG_AUTO_R','MAG_AUTO_I','MAG_AUTO_Z','MAG_AUTO_G-MAG_AUTO_R','MAG_AUTO_G-MAG_AUTO_I',
               'MAG_AUTO_G-MAG_AUTO_Z','MAG_AUTO_R-MAG_AUTO_I','MAG_AUTO_R-MAG_AUTO_Z','MAG_AUTO_I-MAG_AUTO_Z']

inpTrain, train_info = ml.dataSet(d, feats_des, ['ID', 'zb_1', 'Field']).loadData()

inpTest, test_info = ml.dataSet(d2, feats_des, ['ID', 'zb_1', 'Field']).loadData()


In [ ]:
fields = np.unique(d['Field'])
fields = [i for i in fields if i != 4]
print fields


In [ ]:
psudo_des = {}
for i1, fld in enumerate(fields):
    ind_train = np.zeros(len(d), dtype=bool)
    train_fields = [i for i in fields if i != fld]
    for fld1 in train_fields:
        ind_train += d['Field']==fld1
    print 'train_fields', np.unique(d['Field'][ind_train])
    ind_train = np.arange(len(d))[ind_train]
    np.random.shuffle(ind_train)
    
    test_fields = [fld, 4]
    ind_test = np.zeros(len(d), dtype=bool)
    for fld1 in test_fields:
        ind_test += d['Field']==fld1
    print 'test fields', np.unique(d['Field'][ind_test])
    
    train_weights = np.ones(len(ind_train), dtype=float)
    train_weights = train_weights / np.sum(train_weights)
    
    results, score = k_fold_val(inpTrain[ind_train], train_info['zb_1'][ind_train], train_weights, 10, 30)
    print score
    psudo_des[fld] = {}
    psudo_des[fld]['clf'] = results['clf']
    psudo_des[fld]['results'] = results
    psudo_des[fld]['predict_test'] = results['clf'].predict(inpTrain[ind_test])
    psudo_des[fld]['test_ID'] = train_info['ID'][ind_test]
    psudo_des[fld]['test_Field'] = train_info['Field'][ind_test]
    psudo_des[fld]['truth_test'] = train_info['zb_1'][ind_test]
    psudo_des[fld]['predict_test_desphot'] = results['clf'].predict(inpTest)
    psudo_des[fld]['truth_test_des_phot'] = test_info['zb_1']
    psudo_des[fld]['test_ID_des_phot'] = test_info['ID']

In [ ]:

cosmos = '/Users/hoyleb/DATA/DES/PHOTOZ/Y1A1_GOLD_1_0_3_COSMOS2015_matched.cleanmags.fits'
feats_des = ['MAG_AUTO_G','MAG_AUTO_R','MAG_AUTO_I','MAG_AUTO_Z','MAG_AUTO_G-MAG_AUTO_R','MAG_AUTO_G-MAG_AUTO_I',
               'MAG_AUTO_G-MAG_AUTO_Z','MAG_AUTO_R-MAG_AUTO_I','MAG_AUTO_R-MAG_AUTO_Z','MAG_AUTO_I-MAG_AUTO_Z']

inpCos, Cos_info = ml.dataSet(cosmos, feats_des, ['ZMINCHI2', 'COADD_OBJECTS_ID']).loadData()


In [ ]:
psudo_des = pickle.load(open('/Users/hoyleb/DATA/ALHAMBRA/pseudo.des.mags.train5fields.test2.p', 'r'))

In [ ]:

col = ['red', 'blue', 'green', 'purple', 'grey', 'orange','red']
tom_zbins = [0, 0.3, 0.5, 0.7, 0.9, 1.1, 1.3]
zbins = np.linspace(0, 2, num=200)

for fld in psudo_des.keys():
    print 'for field', fld   , '& 4' 
    
    z_ml_ps = psudo_des[fld]['predict_test']
    z_truth_ps = psudo_des[fld]['truth_test']
    
    
    z_ml = psudo_des[fld]['predict_test_desphot']
    z_truth = psudo_des[fld]['truth_test_des_phot']
    
    z_cosmo = psudo_des[fld]['clf'].predict(inpCos)
    z_cosmo_truth = Cos_info['ZMINCHI2']
    
    f = plt.figure()
    for j in range(len(tom_zbins)-1):
        ind = (z_ml_ps <= tom_zbins[j+1]) * (z_ml_ps > tom_zbins[j])
        z_txt= '{:0.2f}<z<{:0.2f}'.format(tom_zbins[j], tom_zbins[j+1])
        print z_txt
        wlres =  wl_val_test(np.mean(z_truth_ps[ind]), np.mean(z_ml_ps[ind]))
        print wlres
      
        pz_ = z_ml_ps[ind]# + np.random.normal(size=np.sum(ind))*sigma[ind]
        train_t_z = gss_kde(pz_).evaluate(zbins) 
        test_t_z = gss_kde(z_truth_ps[ind]).evaluate(zbins) 
    
        plt.plot(zbins, train_t_z, '--',color=col[j], alpha=0.8, linewidth=3, label='ML-z: {:0.2f} '.format(wlres))
        plt.plot(zbins, test_t_z, '-',color=col[j], alpha=0.8, linewidth=3,label='Alh-z: ' + z_txt)
    plt.legend()
    plt.xlabel('redshift')
    plt.ylabel('density')
    plt.title('Field 4&{:} pseudo mags'.format(fld))
    
   
    f = plt.figure()
    for j in range(len(tom_zbins)-1):
        ind = (z_ml <= tom_zbins[j+1]) * (z_ml > tom_zbins[j])
        z_txt= '{:0.2f}<z<{:0.2f}'.format(tom_zbins[j], tom_zbins[j+1])
        print z_txt
        wlres =  wl_val_test(np.mean(z_truth[ind]), np.mean(z_ml[ind]))
        print wlres
      
        pz_ = z_ml[ind]# + np.random.normal(size=np.sum(ind))*sigma[ind]
        train_t_z = gss_kde(pz_).evaluate(zbins) 
        test_t_z = gss_kde(z_truth[ind]).evaluate(zbins) 
    
        plt.plot(zbins, train_t_z, '--',color=col[j], alpha=0.8, linewidth=3, label='ML-z: {:0.2f} '.format(wlres))
        plt.plot(zbins, test_t_z, '-',color=col[j], alpha=0.8, linewidth=3,label='Alh-z: ' + z_txt)
    plt.legend()
    plt.xlabel('redshift')
    plt.ylabel('density')
    plt.title('Field 4 real mags')
    
       
    f = plt.figure()
    for j in range(len(tom_zbins)-1):
        ind = (z_cosmo <= tom_zbins[j+1]) * (z_cosmo > tom_zbins[j])
        z_txt= '{:0.2f}<z<{:0.2f}'.format(tom_zbins[j], tom_zbins[j+1])
        print z_txt
        wlres =  wl_val_test(np.mean(z_cosmo_truth[ind]), np.mean(z_cosmo[ind]))
        print wlres
      
        pz_ = z_cosmo[ind]# + np.random.normal(size=np.sum(ind))*sigma[ind]
        train_t_z = gss_kde(pz_).evaluate(zbins) 
        test_t_z = gss_kde(z_cosmo_truth[ind]).evaluate(zbins) 
    
        plt.plot(zbins, train_t_z, '--',color=col[j], alpha=0.8, linewidth=3, label='ML-z: {:0.2f} '.format(wlres))
        plt.plot(zbins, test_t_z, '-',color=col[j], alpha=0.8, linewidth=3,label='Alh-z: ' + z_txt)
    plt.legend()
    plt.xlabel('redshift')
    plt.ylabel('density')
    plt.title('Cosmos photo-z real mags excl. {:}&4'.format(fld))
   



In [ ]:
col = ['red', 'blue', 'green', 'purple', 'grey', 'orange','red']
tom_zbins = [0, 0.3, 0.5, 0.7, 0.9, 1.1, 1.3]
zbins = np.linspace(0, 2, num=200)

f = plt.figure()
for i, fld in enumerate(psudo_des.keys()):

    z_ml_ps = psudo_des[fld]['predict_test']
    z_truth_ps = psudo_des[fld]['truth_test']

    wlresArr = []
    z_meanArr = []
    for j in range(len(tom_zbins)-1):
        ind = (z_ml_ps <= tom_zbins[j+1]) * (z_ml_ps > tom_zbins[j])
        z_mean = (tom_zbins[j] +  tom_zbins[j+1]) / 2.0
        wlres =  wl_val_test(np.mean(z_truth_ps[ind]), np.mean(z_ml_ps[ind]))
        #print wlres
        if wlres is True:
            wlres = 0.01
        wlresArr.append(wlres)
        z_meanArr.append(z_mean)
        
    plt.plot(z_meanArr, wlresArr, '--',color=col[i], alpha=0.8, linewidth=3, label='Test Field 4&{:}'.format(fld))
plt.legend()
plt.xlabel('redshift')
plt.ylabel('WL: |<z>-<z_ml>|')
plt.title('Pseudo mags')
plt.ylim(0,0.08)



f = plt.figure()
for i, fld in enumerate(psudo_des.keys()):

    z_ml_ps = psudo_des[fld]['predict_test']
    z_truth_ps = psudo_des[fld]['truth_test']
    mu = []
    sig = []
    z_meanArr = []
    for j in range(len(tom_zbins)-1):
        ind = (z_ml_ps <= tom_zbins[j+1]) * (z_ml_ps > tom_zbins[j])
        z_mean = (tom_zbins[j] +  tom_zbins[j+1]) / 2.0
        wlres =  wl_val_test(np.mean(z_truth_ps[ind]), np.mean(z_ml_ps[ind]))
        deltaz_keep =  ml_m.DeltaZ(z_truth_ps[ind], z_ml_ps[ind])
        stats = ml_m.getStats(deltaz_keep)
        z_meanArr.append(z_mean)
        mu.append(stats['median'])
        sig.append(stats['sigma_68'])
    z_meanArr = np.array(z_meanArr) + i*0.01 - 0.02
    sig = np.array(sig)*0.1
    plt.errorbar(z_meanArr, mu, yerr=sig, fmt='o',color=col[i], alpha=0.8, label='Test Field 4&{:}'.format(fld))

plt.legend()
plt.xlabel('redshift')
plt.ylabel('mu, sigma68*0.1')
plt.title('Pseudo mags')




In [ ]:
col = ['red', 'blue', 'green', 'purple', 'grey', 'orange','red']
tom_zbins = [0, 0.3, 0.5, 0.7, 0.9, 1.1, 1.3]
zbins = np.linspace(0, 2, num=200)

f = plt.figure()
for i, fld in enumerate(psudo_des.keys()):

    z_ml_ps = psudo_des[fld]['predict_test'][psudo_des[fld]['test_Field']==4]
    z_truth_ps = psudo_des[fld]['truth_test'][psudo_des[fld]['test_Field']==4]

    wlresArr = []
    z_meanArr = []
    for j in range(len(tom_zbins)-1):
        ind = (z_ml_ps <= tom_zbins[j+1]) * (z_ml_ps > tom_zbins[j])
        z_mean = (tom_zbins[j] +  tom_zbins[j+1]) / 2.0
        wlres =  wl_val_test(np.mean(z_truth_ps[ind]), np.mean(z_ml_ps[ind]))
        #print wlres
        if wlres is True:
            wlres = 0.01
        wlresArr.append(wlres)
        z_meanArr.append(z_mean)
        
    plt.plot(z_meanArr, wlresArr, '--',color=col[i], alpha=0.8, linewidth=3, label='Excl. train 4&{:}'.format(fld))
plt.legend()
plt.xlabel('redshift')
plt.ylabel('WL: |<z>-<z_ml>|')
plt.title('Pseudo mags Field 4')
plt.ylim(0,0.08)



f = plt.figure()
for i, fld in enumerate(psudo_des.keys()):

    z_ml_ps = psudo_des[fld]['predict_test'][psudo_des[fld]['test_Field']==4]
    z_truth_ps = psudo_des[fld]['truth_test'][psudo_des[fld]['test_Field']==4]
    mu = []
    sig = []
    z_meanArr = []
    for j in range(len(tom_zbins)-1):
        ind = (z_ml_ps <= tom_zbins[j+1]) * (z_ml_ps > tom_zbins[j])
        z_mean = (tom_zbins[j] +  tom_zbins[j+1]) / 2.0
        wlres =  wl_val_test(np.mean(z_truth_ps[ind]), np.mean(z_ml_ps[ind]))
        deltaz_keep =  ml_m.DeltaZ(z_truth_ps[ind], z_ml_ps[ind])
        stats = ml_m.getStats(deltaz_keep)
        z_meanArr.append(z_mean)
        mu.append(stats['median'])
        sig.append(stats['sigma_68'])
    z_meanArr = np.array(z_meanArr) + i*0.01 - 0.02
    sig = np.array(sig)*0.1
    plt.errorbar(z_meanArr, mu, yerr=sig, fmt='o',color=col[i], alpha=0.8, label='Excl. train 4&{:}'.format(fld))

plt.legend()
plt.xlabel('redshift')
plt.ylabel('mu, sigma68*0.1')
plt.title('Pseudo mags Field 4')




In [ ]:

f = plt.figure()
for i, fld in enumerate(psudo_des.keys()):
    z_ml = psudo_des[fld]['predict_test_desphot']
    z_truth = psudo_des[fld]['truth_test_des_phot']

    wlresArr = []
    z_meanArr = []
    for j in range(len(tom_zbins)-1):
        ind = (z_ml <= tom_zbins[j+1]) * (z_ml > tom_zbins[j])
        z_mean = (tom_zbins[j] +  tom_zbins[j+1]) / 2.0
        wlres =  wl_val_test(np.mean(z_truth[ind]), np.mean(z_ml[ind]))
        #print wlres
        if wlres is True:
            wlres = 0.01
        wlresArr.append(wlres)
        z_meanArr.append(z_mean)
        
    plt.plot(z_meanArr, wlresArr, '--',color=col[i], alpha=0.8, linewidth=3, label='Excl. train 4&{:}'.format(fld))
plt.legend()
plt.xlabel('redshift')
plt.ylabel('WL: |<z>-<z_ml>|')
plt.title('DES mags field 4')
plt.ylim(0,0.15)



f = plt.figure()
for i, fld in enumerate(psudo_des.keys()):

    z_ml = psudo_des[fld]['predict_test_desphot']
    z_truth = psudo_des[fld]['truth_test_des_phot']

    mu = []
    sig = []
    z_meanArr = []
    for j in range(len(tom_zbins)-1):
        ind = (z_ml <= tom_zbins[j+1]) * (z_ml > tom_zbins[j])
        z_mean = (tom_zbins[j] +  tom_zbins[j+1]) / 2.0
        wlres =  wl_val_test(np.mean(z_truth[ind]), np.mean(z_ml[ind]))
        deltaz_keep =  ml_m.DeltaZ(z_truth[ind], z_ml[ind])
        stats = ml_m.getStats(deltaz_keep)
        z_meanArr.append(z_mean)
        mu.append(stats['median'])
        sig.append(stats['sigma_68'])
    z_meanArr = np.array(z_meanArr) + i*0.01 - 0.02
    sig = np.array(sig)*0.1
    plt.errorbar(z_meanArr, mu, yerr=sig, fmt='o',color=col[i], alpha=0.8, label='Excl. train 4&{:}'.format(fld))

plt.legend()
plt.xlabel('redshift')
plt.ylabel('mu, sigma68*0.1')
plt.title('Real DES mags field 4')



In [ ]:

f = plt.figure()
for i, fld in enumerate(psudo_des.keys()):

    z_cosmo = psudo_des[fld]['clf'].predict(inpCos)
    z_cosmo_truth = Cos_info['ZMINCHI2']
    
    wlresArr = []
    z_meanArr = []
    for j in range(len(tom_zbins)-1):
        ind = (z_cosmo <= tom_zbins[j+1]) * (z_cosmo > tom_zbins[j])
        z_mean = (tom_zbins[j] +  tom_zbins[j+1]) / 2.0
        wlres =  wl_val_test(np.mean(z_cosmo_truth[ind]), np.mean(z_cosmo[ind]))
        #print wlres
        if wlres is True:
            wlres = 0.01
        wlresArr.append(wlres)
        z_meanArr.append(z_mean)
        
    plt.plot(z_meanArr, wlresArr, '--',color=col[i], alpha=0.8, linewidth=3, label='Excl. train 4&{:}'.format(fld))
plt.legend()
plt.xlabel('redshift')
plt.ylabel('WL: |<z>-<z_ml>|')
plt.title('DES mags Cosmos')
plt.ylim(0,0.15)



f = plt.figure()
for i, fld in enumerate(psudo_des.keys()):

    z_cosmo = psudo_des[fld]['clf'].predict(inpCos)
    z_cosmo_truth = Cos_info['ZMINCHI2']

    mu = []
    sig = []
    z_meanArr = []
    for j in range(len(tom_zbins)-1):
        ind = (z_cosmo_truth <= tom_zbins[j+1]) * (z_cosmo > tom_zbins[j])
        z_mean = (tom_zbins[j] +  tom_zbins[j+1]) / 2.0
        wlres =  wl_val_test(np.mean(z_cosmo_truth[ind]), np.mean(z_cosmo[ind]))
        deltaz_keep =  ml_m.DeltaZ(z_cosmo_truth[ind], z_cosmo[ind])
        stats = ml_m.getStats(deltaz_keep)
        z_meanArr.append(z_mean)
        mu.append(stats['median'])
        sig.append(stats['sigma_68'])
    z_meanArr = np.array(z_meanArr) + i*0.01 - 0.02
    sig = np.array(sig)*0.1
    plt.errorbar(z_meanArr, mu, yerr=sig, fmt='o',color=col[i], alpha=0.8, label='Excl. train 4&{:}'.format(fld))

plt.legend()
plt.xlabel('redshift')
plt.ylabel('mu, sigma68*0.1')
plt.title('Real DES mags Cosmos')



In [ ]:
#make ALPHAMBA DES file for BPZ
d = Table.read('/Users/hoyleb/DATA/ALHAMBRA/ALHAMBRA_IN_DES.BPZ.Yv103.csv.fits')
d = d[d['MAG_AUTO_I']<23.5]
d = d[d['MAG_AUTO_I']>14]
d = d[d['MAGERR_AUTO_I']<0.5]
d = d[d['MAG_AUTO_G']<30]
d = d[d['MAG_AUTO_I']<30]
d = d[d['MAG_AUTO_R']<30]
d = d[d['MAG_AUTO_Z']<30]

In [ ]:
f= open('/Users/hoyleb/Documents/third_party/BPZ/alhambra/y1a1_final_spec_valid.cat','w')
f.write('# COADD_OBJECTS_ID\tMAG_AUTO_G\tMAG_AUTO_R\tMAG_AUTO_I\tMAG_AUTO_Z\tMAGERR_AUTO_G\tMAGERR_AUTO_R\tMAGERR_AUTO_I\tMAGERR_AUTO_Z\tZ\n')

keys=['COADD_OBJECTS_ID','MAG_AUTO_G','MAG_AUTO_R','MAG_AUTO_I','MAG_AUTO_Z','MAGERR_AUTO_G',
      'MAGERR_AUTO_R','MAGERR_AUTO_I','MAGERR_AUTO_Z','ZB_1']

for i in range(len(d)):
    dic = {}
    for j in keys:
        dic[j] = d[j][i]
    if np.isfinite(d['MAG_AUTO_R'][i]):
        f.write('{COADD_OBJECTS_ID}\t{MAG_AUTO_G:0.6}\t{MAG_AUTO_R:0.6}\t{MAG_AUTO_I:0.6}\t{MAG_AUTO_Z:0.6}\t{MAGERR_AUTO_G:0.6}\t{MAGERR_AUTO_R:0.6}\t{MAGERR_AUTO_I:0.6}\t{MAGERR_AUTO_Z:0.6}\t{ZB_1}\n'.format(**dic))

f.close()

In [ ]:
d1 = np.genfromtext('/Users/hoyleb/Documents/third_party/BPZ/alhambra/y1a1_final_spec_valid.bpz')

<h3>Test alhambra photometry is similar to DES Y1</h3>

In [ ]:
features = ['MAG_AUTO_I', 'MAG_AUTO_G','MAG_AUTO_R', 'MAG_AUTO_Z', 'MAG_AUTO_G-MAG_AUTO_R', 
            'MAG_AUTO_R-MAG_AUTO_I', 'MAG_AUTO_Z-MAG_AUTO_I', 
            'MAG_AUTO_Z-MAG_AUTO_G', 'MAGERR_AUTO_I']
outF = ['COADD_OBJECTS_ID','MAG_AUTO_I', 'MAG_AUTO_G','MAG_AUTO_R', 
                                                      'MAG_AUTO_Z', 'MAGERR_AUTO_I']

In [ ]:

inputs_alh, info_alh = ml.dataSet('/Users/hoyleb/DATA/DES/PHOTOZ/Y1v103/ALHCOSVALIDATIONID.BPZ.WLflags.Y1v103.fits', 
                                           features, outF).loadData()
ind = (info_alh['MAG_AUTO_I']<23.5)*(info_alh['MAG_AUTO_I']>17) *(info_alh['MAGERR_AUTO_I']<0.3)*(info_alh['MAGERR_AUTO_I']>0)*(info_alh['MAG_AUTO_G']<30)*(info_alh['MAG_AUTO_G']>15)*(info_alh['MAG_AUTO_R']<30)*(info_alh['MAG_AUTO_R']>15)*(info_alh['MAG_AUTO_I']<30)*(info_alh['MAG_AUTO_I']>15)*(info_alh['MAG_AUTO_Z']<30)*(info_alh['MAG_AUTO_Z']>15)
inputs_alh = inputs_alh[ind]

ind = np.ones(len(inputs_alh), dtype=bool)
#for i in np.arange(len(inputs_alh[0])-4)+4:
#    ind *= (inputs_alh[:, i]>-6) * (inputs_alh[:, i]<2)
#inputs_alh = inputs_alh[ind]

inputs_y1, info_y1 = ml.dataSet('/Users/hoyleb/DATA/DES/PHOTOZ/WL_INFO_0_1M_ID.Y1A1_GOLD_1_0_3.nside128.fits', 
                                           features,outF + ['RA']).loadData()
ind = (info_y1['MAG_AUTO_I']<23.5)*(info_y1['MAG_AUTO_I']>17) *(info_y1['MAGERR_AUTO_I']<0.3)*(
    info_y1['MAGERR_AUTO_I']>0)*(info_y1['MAG_AUTO_G']<30)*(info_y1['MAG_AUTO_G']>15)*(info_y1['MAG_AUTO_R']<30)*(
    info_y1['MAG_AUTO_R']>15)*(info_y1['MAG_AUTO_I']<30)*(info_y1['MAG_AUTO_I']>15)*(
    info_y1['MAG_AUTO_Z']<30)*(info_y1['MAG_AUTO_Z']>15)*(info_y1['RA']>45)
inputs_y1 = inputs_y1[ind]

inputs_sn1, info_sn1 = ml.dataSet('/Users/hoyleb/DATA/DES/PHOTOZ/Sne0.5Deg.fits', 
                                           features,outF).loadData()
ind = (info_sn1['MAG_AUTO_I']<23.5)*(info_sn1['MAG_AUTO_I']>17) *(info_sn1['MAGERR_AUTO_I']<0.3)*(
    info_sn1['MAGERR_AUTO_I']>0)*(info_sn1['MAG_AUTO_G']<30)*(info_sn1['MAG_AUTO_G']>15)*(info_sn1['MAG_AUTO_R']<30)*(
    info_sn1['MAG_AUTO_R']>15)*(info_sn1['MAG_AUTO_I']<30)*(info_sn1['MAG_AUTO_I']>15)*(
    info_sn1['MAG_AUTO_Z']<30)*(info_sn1['MAG_AUTO_Z']>15)
inputs_sn1 = inputs_sn1[ind]

ind = np.ones(len(inputs_y1), dtype=bool)
#for i in np.arange(len(inputs_y1[0])-4)+4:
#    ind *= (inputs_y1[:, i]>-6) * (inputs_y1[:, i]<2)
#inputs_y1 = inputs_y1[ind]

ind1 = np.arange(len(inputs_y1))
np.random.shuffle(ind1)
ind1 = ind1[0: len(inputs_alh)]

In [ ]:
from scipy import stats


for i in range(len(features)):
    print features[i], stats.ks_2samp(inputs_y1[ind1, i], inputs_alh[:, i])
    f = plt.figure()
    
    _ = plt.hist(inputs_alh[:, i], bins=100, label='Cosmos[Alh] footprint', normed=True, alpha=0.5)
    _ = plt.hist(inputs_y1[ind1, i], bins=100, label='WL objects', normed=True, alpha=0.5)
    _ = plt.hist(inputs_sn1[:, i], bins=100, label='SNe 0.5deg', normed=True, alpha=0.5)
    plt.xlabel(features[i])
    plt.legend()
    plt.xlim(14,30)
    

In [ ]:
plt.plot(inputs_alh[:, 0], inputs_alh[:, -1],',', label='Cosmos[Alh] footprint', alpha=1)
plt.plot(inputs_y1[:, 0],inputs_y1[:, -1], ',', label='WL galaxies', alpha=1)
plt.plot(inputs_sn1[:, 0],inputs_sn1[:, -1], ',', label='Sne 0.5deg', alpha=1)

plt.xlabel(features[0])
plt.ylabel(features[-1])
plt.legend(loc=2)
plt.yscale('log')